#01_EDA

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib


import sklearn
import xgboost
from xgboost import XGBRegressor


from sklearn.model_selection import KFold

import random as rn

RANDOM_SEED = 2025
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)



from datetime import datetime

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
# 한글 폰트 관련
# 1. 나눔 폰트 설치
!apt-get -y install fonts-nanum

# 2. 런타임 재시작 없이 폰트 강제 등록
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib as mpl

# 3. 폰트 경로 직접 지정
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
fm.fontManager.addfont(font_path)
nanum_font = fm.FontProperties(fname=font_path).get_name()

# 4. matplotlib 기본 설정에 강제로 반영
mpl.rcParams['font.family'] = nanum_font
mpl.rcParams['axes.unicode_minus'] = False


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (6,883 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/dacon_2025_Power_usage_prediction_AI_github/2025dacon_powerU_github

/content/drive/MyDrive/dacon_2025_Power_usage_prediction_AI_github/2025dacon_powerU_github


In [5]:
!git config --global user.name "SonHyesang0914"
!git config --global user.email "hyesang0914@gmail.com"

##02-6. 건물별 예상 휴일 추가 (개인 판단)

In [6]:
train_preprocessed_path = "/content/drive/MyDrive/dacon_2025_Power_usage_prediction_AI/dacon_data/train_preprocessed.csv"
test_preprocessed_path = "/content/drive/MyDrive/dacon_2025_Power_usage_prediction_AI/dacon_data/test_preprocessed.csv"
train_preprocessed = pd.read_csv(train_preprocessed_path)
test_preprocessed = pd.read_csv(test_preprocessed_path)

In [7]:
# date_time을 datetime으로 변환
train_preprocessed['date_time'] = pd.to_datetime(train_preprocessed['date_time'])
test_preprocessed['date_time'] = pd.to_datetime(test_preprocessed['date_time'])


In [8]:
# 1) 우선 모든 행을 0으로 초기화
train_preprocessed['holiday_by_building'] = 0
test_preprocessed['holiday_by_building']  = 0

# 2) 휴일로 판단한 날짜 목록 정의
#    형식: {건물번호: [(월, 일), (월, 일), ...], ...}
manual_holidays_test = {
    3: [(8, 25)],
    5: [(8, 25)],
    6: [(8, 25)],
    8: [(8, 25)],
    12: [(8, 25)],
    13: [(8, 25)],
    14: [(8, 25)],
    15: [(8, 25)],
    16: [(8, 25)],
    17: [(8, 25)],
    20: [(8, 25)],
    21: [(8, 25)],
    22: [(8, 25)],
    23: [(8, 25)],
    24: [(8, 25)],
    27: [(8, 25)],
    29: [(8, 25)],
    32: [(8, 26)],
    37: [(8, 25)],
    38: [(8, 25)],
    39: [(8, 25)],
    40: [(8, 25)],
    42: [(8, 25)],
    43: [(8, 25)],
    44: [(8, 25)],
    46: [(8, 25)],
    47: [(8, 25)],
    48: [(8, 25)],
    49: [(8, 25)],
    50: [(8, 25)],
    51: [(8, 25)],
    52: [(8, 25)],
    53: [(8, 25)],
    55: [(8, 25)],
    59: [(8, 25)],
    60: [(8, 25)],
    62: [(8, 25)],
    63: [(8, 25)],
    64: [(8, 25)],
    66: [(8, 25)],
    67: [(8, 25)],
    68: [(8, 25)],
    69: [(8, 25)],
    72: [(8, 25)],
    74: [(8, 26)],
    75: [(8, 25)],
    80: [(8, 25)],
    83: [(8, 25)],
    86: [(8, 25)],
    87: [(8, 25)],
    90: [(8, 25)],
    94: [(8, 25)],
}
# 2) 휴일로 판단한 날짜 목록 정의
#    형식: {건물번호: [(월, 일), (월, 일), ...], ...}
manual_holidays_train = {
    2: [(6, 1), (6, 8), (6, 15), (6, 22), (6, 29), (7, 6), (7, 13), (7, 20), (7, 27), (8, 3), (8, 10), (8, 17), (8, 24)],
    3: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    5: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    6: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    7: [(7, 7), (7, 8)],
    8: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    12: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    13: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    14: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    15: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    16: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    17: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    18: [(6, 11), (7, 17), (8, 8)],
    19: [(6, 10), (7, 8), (8, 19)],
    20: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    21: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    22: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    23: [(6, 1), (6, 2), (6, 6), (6, 7), (6, 8), (6, 9), (6, 15), (6, 16), (6, 21), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 12), (7, 13), (7, 14), (7, 19), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 16), (8, 17), (8, 18), (8, 23), (8, 24)],
    24: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    27: [(6, 9), (6, 23), (7, 14), (7, 28), (8, 11)],
    29: [(6, 10), (6, 23), (7, 10), (7, 28), (8, 10)],
    32: [(6, 10), (6, 24), (7, 8), (7, 22), (8, 12)],
    37: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    38: [(6, 1), (6, 2), (6, 6), (6, 7), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    39: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    40: [(6, 9), (6, 23), (7, 14), (7, 28), (8, 11)],
    42: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    43: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    44: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    45: [(6, 10), (7, 8), (8, 19)],
    46: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    47: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    48: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    49: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 22), (8, 24)],
    50: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    51: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    52: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 28), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    53: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    54: [(6, 17), (7, 1), (8, 19)],
    55: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    59: [(6, 9), (6, 23), (7, 14), (7, 28), (8, 11)],
    60: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    62: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    63: [(6, 9), (6, 23), (7, 14), (7, 28), (8, 11)],
    64: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    66: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    67: [(6, 1), (6, 2), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    68: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    69: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    72: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    74: [(6, 17), (7, 1)],
    75: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    79: [(6, 17), (7, 1), (8, 19)],
    80: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    83: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    86: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    87: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    90: [(6, 1), (6, 2), (6, 6), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (7, 27), (7, 28), (8, 3), (8, 4), (8, 10), (8, 11), (8, 15), (8, 17), (8, 18), (8, 24)],
    94: [(6, 1), (6, 2), (6, 6), (6, 7), (6, 8), (6, 9), (6, 15), (6, 16), (6, 22), (6, 23), (6, 29), (6, 30), (7, 6), (7, 7), (7, 13), (7, 14), (7, 20), (7, 21), (8, 10), (8, 11), (8, 15), (8, 16), (8, 17), (8, 18), (8, 24)],
    95: [(7, 8), (7, 5)]
}


# 3) train에 반영
for bld, dates in manual_holidays_train.items():
    for mm, dd in dates:
        mask = (train_preprocessed['building_num'] == bld) & (train_preprocessed['month'] == mm) & (train_preprocessed['day'] == dd)
        train_preprocessed.loc[mask, 'holiday_by_building'] = 1

# 4) test에도 동일 규칙 반영
for bld, dates in manual_holidays_test.items():
    for mm, dd in dates:
        mask = (test_preprocessed['building_num'] == bld) & (test_preprocessed['month'] == mm) & (test_preprocessed['day'] == dd)
        test_preprocessed.loc[mask, 'holiday_by_building'] = 1


In [9]:
for day in range(13, 32):
    mask = (train_preprocessed['building_num'] == 7) & (train_preprocessed['month'] == 7) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1


In [10]:
for day in range(1, 6):
    mask = (train_preprocessed['building_num'] == 7) & (train_preprocessed['month'] == 8) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1



In [11]:
for day in range(25, 31):

    mask_test = (test_preprocessed['building_num'] == 10) & (test_preprocessed['month'] == 8) & (test_preprocessed['day'] == day)
    test_preprocessed.loc[mask_test, 'holiday_by_building'] = 1

In [12]:
for day in range(1, 31):
    mask = (train_preprocessed['building_num'] == 10) & (train_preprocessed['month'] == 6) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1

In [13]:
for day in range(1, 4):
    mask = (train_preprocessed['building_num'] == 10) & (train_preprocessed['month'] == 7) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1

In [14]:
for day in range(1, 31):
    mask = (train_preprocessed['building_num'] == 11) & (train_preprocessed['month'] == 6) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1



In [15]:
for day in range(1, 20):
    mask = (train_preprocessed['building_num'] == 11) & (train_preprocessed['month'] == 7) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1



In [16]:
for day in range(27, 32):
    mask = (train_preprocessed['building_num'] == 94) & (train_preprocessed['month'] == 7) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1


In [17]:
for day in range(1, 5):
    mask = (train_preprocessed['building_num'] == 94) & (train_preprocessed['month'] == 8) & (train_preprocessed['day'] == day)
    train_preprocessed.loc[mask, 'holiday_by_building'] = 1

##02-7. 성수기 여부 (7/10- ~8/25 예상)

In [18]:
# -----------------------------
# 1) date_time을 datetime으로 변환
# -----------------------------
for df in [train_preprocessed, test_preprocessed]:
    if "date_time" in df.columns:
        df["date_time"] = pd.to_datetime(df["date_time"])

# -----------------------------
# 2) 성수기 feature 추가
# -----------------------------
def add_peak_summer_feature(df):
    # 2024년 성수기 기간
    start = pd.Timestamp("2024-07-10")
    end   = pd.Timestamp("2024-08-25")

    df["is_peak_summer"] = df["date_time"].between(start, end).astype(int)
    return df

train_preprocessed = add_peak_summer_feature(train_preprocessed)
test_preprocessed = add_peak_summer_feature(test_preprocessed)

# -----------------------------
# 3) 확인
# -----------------------------
print(train_preprocessed[["date_time", "is_peak_summer"]].head(40))


             date_time  is_peak_summer
0  2024-06-01 00:00:00               0
1  2024-06-01 01:00:00               0
2  2024-06-01 02:00:00               0
3  2024-06-01 03:00:00               0
4  2024-06-01 04:00:00               0
5  2024-06-01 05:00:00               0
6  2024-06-01 06:00:00               0
7  2024-06-01 07:00:00               0
8  2024-06-01 08:00:00               0
9  2024-06-01 09:00:00               0
10 2024-06-01 10:00:00               0
11 2024-06-01 11:00:00               0
12 2024-06-01 12:00:00               0
13 2024-06-01 13:00:00               0
14 2024-06-01 14:00:00               0
15 2024-06-01 15:00:00               0
16 2024-06-01 16:00:00               0
17 2024-06-01 17:00:00               0
18 2024-06-01 18:00:00               0
19 2024-06-01 19:00:00               0
20 2024-06-01 20:00:00               0
21 2024-06-01 21:00:00               0
22 2024-06-01 22:00:00               0
23 2024-06-01 23:00:00               0
24 2024-06-02 00:00:00   

##02-8. 1주일 전 사용량

In [19]:
# train과 test를 building_num별로 처리
def add_weekly_lag_feature(df, target_col='power_consumption', lag_hours=168):
    df = df.sort_values(['building_num', 'date_time']).copy()
    df[f'lag_{lag_hours}h'] = (
        df.groupby('building_num')[target_col].shift(lag_hours)
    )
    return df

# train에는 power_consumption이 있으므로 바로 가능
train_preprocessed = add_weekly_lag_feature(train_preprocessed, target_col='power_consumption', lag_hours=168)

# test_preprocessed에는 실제 'power_consumption'이 없으므로,
# train_preprocessed과 이어붙여서 shift 후 분리하는 방식 필요
combined = pd.concat([train_preprocessed[['building_num','date_time','power_consumption']],
                      test_preprocessed[['building_num','date_time']]],
                     axis=0, ignore_index=True)

combined = combined.sort_values(['building_num','date_time'])

combined['lag_168h'] = (
    combined.groupby('building_num')['power_consumption'].shift(168)
)

# 다시 train/test로 분리
train_preprocessed['lag_168h'] = combined.loc[combined['date_time'].isin(train_preprocessed['date_time']), 'lag_168h'].values
test_preprocessed['lag_168h']  = combined.loc[combined['date_time'].isin(test_preprocessed['date_time']),  'lag_168h'].values


In [20]:
# 전처리 끝난 DataFrame 이름이 train_final, test_final 이라고 가정
out_dir = "/content/drive/MyDrive/dacon_2025_Power_usage_prediction_AI/dacon_data/"

# train 저장
train_preprocessed.to_csv(out_dir + "train_preprocessed4.csv", index=False)

# test 저장
test_preprocessed.to_csv(out_dir + "test_preprocessed4.csv", index=False)

In [23]:
train_preprocessed

,building_num,date_time,temperature,rainfall,windspeed,humidity,power_consumption,building_type,total_area,cooling_area,solar_power_include,ess_include,hour,day,month,day_of_week,CDH,THI,WCT,day_max_temperature,day_mean_temperature,day_min_temperature,day_temperature_range,doy,sin_hour,cos_hour,sin_dayofweek,cos_dayofweek,sin_doy,cos_doy,sin_month,cos_month,is_weekend,is_holiday,holiday_by_building,is_peak_summer,lag_168h
0,1,2024-06-01 00:00:00,18.3,0.0,2.6,82.0,5794.80,Hotel,82912.71,77586.0,0,0,0,1,6,5,0.0,17.92380,18.609436,24.8,20.570833,17.6,7.2,153,0.000000,1.000000,-0.974928,-0.222521,0.492548,-0.870285,1.224647e-16,-1.0,1,0,0,0,NaN
1,1,2024-06-01 01:00:00,18.3,0.0,2.7,82.0,5591.85,Hotel,82912.71,77586.0,0,0,1,1,6,5,0.0,17.92380,18.573798,24.8,20.570833,17.6,7.2,153,0.258819,0.965926,-0.974928,-0.222521,0.492548,-0.870285,1.224647e-16,-1.0,1,0,0,0,NaN
2,1,2024-06-01 02:00:00,18.1,0.0,2.6,80.0,5338.17,Hotel,82912.71,77586.0,0,0,2,1,6,5,0.0,17.70400,18.371719,24.8,20.570833,17.6,7.2,153,0.500000,0.866025,-0.974928,-0.222521,0.492548,-0.870285,1.224647e-16,-1.0,1,0,0,0,NaN
3,1,2024-06-01 03:00:00,18.0,0.0,2.6,81.0,4554.42,Hotel,82912.71,77586.0,0,0,3,1,6,5,0.0,17.63425,18.252861,24.8,20.570833,17.6,7.2,153,0.707107,0.707107,-0.974928,-0.222521,0.492548,-0.870285,1.224647e-16,-1.0,1,0,0,0,NaN
4,1,2024-06-01 04:00:00,17.8,0.0,1.3,81.0,3602.25,Hotel,82912.71,77586.0,0,0,4,1,6,5,0.0,17.45515,18.662583,24.8,20.570833,17.6,7.2,153,0.866025,0.500000,-0.974928,-0.222521,0.492548,-0.870285,1.224647e-16,-1.0,1,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203851,100,2024-08-24 19:00:00,29.1,0.0,4.4,76.0,3276.00,Hotel,162070.24,152943.0,0,0,19,24,8,5,82.9,27.17280,31.467262,32.9,29.454167,26.8,6.1,237,-0.965926,0.258819,-0.974928,-0.222521,-0.799839,-0.600214,-8.660254e-01,-0.5,1,0,0,1,2667.72
203852,100,2024-08-24 20:00:00,28.6,0.0,3.7,74.0,3197.52,Hotel,162070.24,152943.0,0,0,20,24,8,5,82.9,26.58370,30.849350,32.9,29.454167,26.8,6.1,237,-0.866025,0.500000,-0.974928,-0.222521,-0.799839,-0.600214,-8.660254e-01,-0.5,1,0,0,1,1994.88
203853,100,2024-08-24 21:00:00,28.3,0.0,2.9,74.0,3006.60,Hotel,162070.24,152943.0,0,0,21,24,8,5,83.1,26.32660,30.491518,32.9,29.454167,26.8,6.1,237,-0.707107,0.707107,-0.974928,-0.222521,-0.799839,-0.600214,-8.660254e-01,-0.5,1,0,0,1,2177.76
203854,100,2024-08-24 22:00:00,28.0,0.0,1.7,76.0,2649.72,Hotel,162070.24,152943.0,0,0,22,24,8,5,83.1,26.21800,30.163891,32.9,29.454167,26.8,6.1,237,-0.500000,0.866025,-0.974928,-0.222521,-0.799839,-0.600214,-8.660254e-01,-0.5,1,0,0,1,2911.32


In [21]:
!git status
!git add 02_EDAandPreprocessing_2.ipynb
!git commit -m "2차 전처리-5(1주일 전 사용량 추가)"
!git push origin main

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   01_EDA.ipynb
	modified:   02_EDAandPreprocessing.ipynb
	modified:   02_EDAandPreprocessing_2.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	03_Modeling_test.ipynb
	03_Modeling_test_GBM_XG.ipynb
	03_Modeling_test_LGBM.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
[main b4f9c75] 2차 전처리-5(1주일 전 사용량 추가)
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite 02_EDAandPreprocessing_2.ipynb (81%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 1.20 KiB | 153.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 1